In [1]:
import optuna
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

In [2]:
data = pd.read_csv('data/_train_sem09.csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [3]:
x = data.drop('Activity', axis=1)
y = data['Activity']

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y, random_state=42)

In [5]:
# Инициализируем модель логистической регрессии и считаем целевую метрику на тесте
logistig_regression = LogisticRegression(max_iter=50, random_state=42)
logistig_regression.fit(x_train, y_train)
y_train_predict = logistig_regression.predict(x_train)
y_test_predict = logistig_regression.predict(x_test)
print(round(metrics.f1_score(y_train, y_train_predict), 2)) # считаем целевую метрику на трейне для отслеживания переобучения
print(round(metrics.f1_score(y_test, y_test_predict), 2))

0.88
0.78


C:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [6]:
# Инициализируем модель случайного леса с рандомными гиперпараметрами и считаем целевую метрику на тесте
random_forest = ensemble.RandomForestClassifier(n_estimators=50, max_depth=7, min_samples_leaf=5, random_state=42)
random_forest.fit(x_train, y_train)
y_train_predict = random_forest.predict(x_train)
y_test_predict = random_forest.predict(x_test)
print(round(metrics.f1_score(y_train, y_train_predict), 2)) # считаем целевую метрику на трейне для отслеживания переобучения
print(round(metrics.f1_score(y_test, y_test_predict), 2))

0.87
0.77


По результатам целевых метрик на тесте двух моделей машинного обучения - логистической регрессии и случайного леса мы видим, что метрика довольно высока - 0.78 и 0.77 соответственно. Причём метрика случайного леса с нашими рандомными гиперпараметрами даже немного ниже метрики случайной регрессии, поэтому попробуем подобрать наиболее оптимальные гиперпараметры, чтобы улучшить нашу метрику.

In [7]:
# Инициализируем сетку параметров для логистической регрессии
param_grid_lr = [
              {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регуляризации
              
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]

In [8]:
grid_search_lr = GridSearchCV(
    estimator=LogisticRegression(max_iter=50, random_state=42),
    param_grid=param_grid_lr,
    cv=5,
    n_jobs=-1)

grid_search_lr.fit(x_train, y_train)
y_train_predict = grid_search_lr.predict(x_train)
y_test_predict = grid_search_lr.predict(x_test)
print(round(metrics.f1_score(y_train, y_train_predict), 2)) # считаем целевую метрику на трейне для отслеживания переобучения
print(round(metrics.f1_score(y_test, y_test_predict), 2))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_lr.best_params_))

0.84
0.78
Наилучшие значения гиперпараметров: {'C': 0.3, 'penalty': 'l1', 'solver': 'saga'}


C:\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [9]:
# Инициализируем сетку параметров для рандомного леса
param_grid_rf = {'n_estimators': list(range(10, 50, 5)),
              'min_samples_leaf': [5],
              'max_depth': list(np.linspace(20, 40, 5, dtype=int))
              }

In [10]:
grid_search_rf = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42),
    param_grid=param_grid_rf,
    cv=5,
    n_jobs=-1)

grid_search_rf.fit(x_train, y_train)
y_train_predict = grid_search_rf.predict(x_train)
y_test_predict = grid_search_rf.predict(x_test)
print(round(metrics.f1_score(y_train, y_train_predict), 2)) # считаем целевую метрику на трейне для отслеживания переобучения
print(round(metrics.f1_score(y_test, y_test_predict), 2))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_rf.best_params_))

0.94
0.78
Наилучшие значения гиперпараметров: {'max_depth': 25, 'min_samples_leaf': 5, 'n_estimators': 45}


С использованием метода GridSearchCV и наших сеток параметров целевая метрика на тесте не улучшилась. Либо мы подобрали неоптимальные сетки, либо и в модели логистической регрессии, и в модели случайного леса по умолчанию подобраны довольно оптимальные гиперпараметры. Причём в модели случайного леса значительно увеличилась метрика на трейне, это может говорить о том, что наша модель значительно переобучена.

In [11]:
randomized_search_lr = RandomizedSearchCV(
    estimator=LogisticRegression(max_iter=50, random_state=42),
    param_distributions=param_grid_lr,
    cv=5,
    n_iter=50,
    n_jobs=-1
    )

randomized_search_lr.fit(x_train, y_train)
y_train_predict = randomized_search_lr.predict(x_train)
y_test_predict = randomized_search_lr.predict(x_test)
print(round(metrics.f1_score(y_train, y_train_predict), 2)) # считаем целевую метрику на трейне для отслеживания переобучения
print(round(metrics.f1_score(y_test, y_test_predict), 2))
print("Наилучшие значения гиперпараметров: {}".format(randomized_search_lr.best_params_))

0.84
0.78
Наилучшие значения гиперпараметров: {'solver': 'saga', 'penalty': 'l1', 'C': 0.3}


C:\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [12]:
randomized_search_rf = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42),
    param_distributions=param_grid_rf,
    cv=5,
    n_iter=50,
    n_jobs=-1
    )

randomized_search_rf.fit(x_train, y_train)
y_train_predict = randomized_search_rf.predict(x_train)
y_test_predict = randomized_search_rf.predict(x_test)
print(round(metrics.f1_score(y_train, y_train_predict), 2)) # считаем целевую метрику на трейне для отслеживания переобучения
print(round(metrics.f1_score(y_test, y_test_predict), 2))
print("Наилучшие значения гиперпараметров: {}".format(randomized_search_rf.best_params_))

C:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 40 is smaller than n_iter=50. Running 40 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


0.94
0.78
Наилучшие значения гиперпараметров: {'n_estimators': 45, 'min_samples_leaf': 5, 'max_depth': 25}


С использованием метода RandomizedSearchCV и наших сеток параметров целевая метрика на тесте не улучшилась. Либо мы подобрали неоптимальные сетки, либо и в модели логистической регрессии, и в модели случайного леса по умолчанию подобраны довольно оптимальные гиперпараметры. Причём в модели случайного леса значительно увеличилась метрика на трейне, это может говорить о том, что наша модель значительно переобучена.

In [13]:
# Зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 10, 50, 1),
       'max_depth' : hp.quniform('max_depth', 20, 40, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 3, 7, 1)
      }

In [14]:
# Инициализируем функцию Хайперопт для подсчёта метрики на модели случайного леса
def hyperopt_rf(params, cv=5, X=x_train, y=y_train, random_state=42):
    # Функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
    # Используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=42)
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))

    # Метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [15]:
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации
          max_evals=50, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(42) # фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|███████████████████████████████████████████████| 50/50 [00:58<00:00,  1.17s/trial, best loss: -0.9713168187744459]
Наилучшие значения гиперпараметров {'max_depth': 39.0, 'min_samples_leaf': 3.0, 'n_estimators': 39.0}


In [16]:
model = ensemble.RandomForestClassifier(
    random_state=42, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(x_train, y_train)
y_train_predict = model.predict(x_train)
y_test_predict = model.predict(x_test)
print(round(metrics.f1_score(y_train, y_train_predict), 2)) # считаем целевую метрику на трейне для отслеживания переобучения
print(round(metrics.f1_score(y_test, y_test_predict), 2))

0.97
0.8


С использованием метода Hyperopt и нашего пространства поиска параметров моделью случайного леса целевая метрика на тесте незначительно улучшилась. Но также увеличилась и метрика на трейне, а это может говорить о том, что наша модель значительно переобучена.

In [17]:
# Инициализируем функцию Оптуна для подсчёта метрики на модели случайного леса
def optuna_rf(trial):
    # Задаём пространства поиска гиперпараметров
    n_estimators = trial.suggest_int('n_estimators', 10, 50, 1)
    max_depth = trial.suggest_int('max_depth', 20, 40, 1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 3, 7, 1)

    model = ensemble.RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        random_state=42)

    model.fit(x_train, y_train)
    score = metrics.f1_score(y_train, model.predict(x_train))

    return score

In [18]:
# Создаём объект исследования и напрямую указываем, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# Ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=50)
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))

[I 2024-04-05 22:43:34,932] A new study created in memory with name: RandomForestClassifier
[I 2024-04-05 22:43:35,545] Trial 0 finished with value: 0.9600000000000001 and parameters: {'n_estimators': 12, 'max_depth': 26, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.9600000000000001.
[I 2024-04-05 22:43:36,910] Trial 1 finished with value: 0.9439374185136896 and parameters: {'n_estimators': 43, 'max_depth': 27, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.9600000000000001.
[I 2024-04-05 22:43:38,571] Trial 2 finished with value: 0.9220862846104314 and parameters: {'n_estimators': 46, 'max_depth': 23, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.9600000000000001.
[I 2024-04-05 22:43:40,088] Trial 3 finished with value: 0.9694408322496749 and parameters: {'n_estimators': 49, 'max_depth': 32, 'min_samples_leaf': 3}. Best is trial 3 with value: 0.9694408322496749.
[I 2024-04-05 22:43:41,220] Trial 4 finished with value: 0.9204508856682769 and parameters: {'n_esti

[I 2024-04-05 22:44:26,753] Trial 38 finished with value: 0.9704257393565161 and parameters: {'n_estimators': 43, 'max_depth': 26, 'min_samples_leaf': 3}. Best is trial 26 with value: 0.9710569105691057.
[I 2024-04-05 22:44:28,172] Trial 39 finished with value: 0.9710380735437684 and parameters: {'n_estimators': 43, 'max_depth': 23, 'min_samples_leaf': 3}. Best is trial 26 with value: 0.9710569105691057.
[I 2024-04-05 22:44:29,554] Trial 40 finished with value: 0.9568041571938941 and parameters: {'n_estimators': 35, 'max_depth': 22, 'min_samples_leaf': 4}. Best is trial 26 with value: 0.9710569105691057.
[I 2024-04-05 22:44:31,135] Trial 41 finished with value: 0.9704257393565161 and parameters: {'n_estimators': 43, 'max_depth': 26, 'min_samples_leaf': 3}. Best is trial 26 with value: 0.9710569105691057.
[I 2024-04-05 22:44:32,636] Trial 42 finished with value: 0.9704257393565161 and parameters: {'n_estimators': 43, 'max_depth': 26, 'min_samples_leaf': 3}. Best is trial 26 with value: 

Наилучшие значения гиперпараметров {'n_estimators': 41, 'max_depth': 20, 'min_samples_leaf': 3}


In [19]:
model = ensemble.RandomForestClassifier(**study.best_params, random_state=42)
model.fit(x_train, y_train)
y_train_predict = model.predict(x_train)
y_test_predict = model.predict(x_test)
print(round(metrics.f1_score(y_train, y_train_predict), 2)) # считаем целевую метрику на трейне для отслеживания переобучения
print(round(metrics.f1_score(y_test, y_test_predict), 2))

0.97
0.79


С использованием метода Optuna и нашего пространства поиска параметров моделью случайного леса целевая метрика на тесте не улучшилась. Метрика на трейне также не изменилась, а это может говорить о том, что наша модель значительно переобучена.

Вывод: целью нашей работы было спрогнозировать биологический ответ. Судя по целевой метрике, у нас это получилось очень хорошо. В процессе работы мы не просто построили и обучили несколько моделей машинного обучения (логистическую регрессию и случайный лес), но и попытались подобрать к ним оптимальные гиперпараметры четырьмя различными способами (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna). Таким образом, биологический ответ спрогнозирован с высокой долей вероятности, методы оптимизации гиперпараметров изучены, работа выполнена на отлично!